<a href="https://colab.research.google.com/github/Leonardo-Dias-Up/API-CVM/blob/main/SAR_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesse script, carregamos o arquivo de dados MUNIC_2020.shp, selecionamos as variáveis de interesse, renomeamos as variáveis, definimos a variável dependente e as independentes, criamos uma matriz de pesos espaciais utilizando a rainha como esquema de contiguidade, normalizamos a matriz de pesos espaciais, ajustamos um modelo de regressão espacial de autorregressão espacial com lag generalizado (GM-Lag) e apresentamos os resultados. Além disso, calculamos a estatística de Moran's I para o erro e para o resíduo e realizamos o teste de heterocedasticidade.


In [ ]:
!pip install geopandas
!pip install pysal

In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
from libpysal import weights
from libpysal import examples
import matplotlib.pyplot as plt
import seaborn as sns
import splot
from pysal.explore import esda

In [ ]:
# Carregar o arquivo de dados
data = gpd.read_file("MUNIC_2020.shp")
data.head()

In [ ]:
# Ajustar os nomes das variáveis
data.columns = ["CD_GEOCMU", "geometry", "pib", "idhm", "pop"]

In [ ]:
# Selecionar as variáveis
data = data[["CD_GEOCMU", "geometry", "PIB", "IDHM", "Pop_residente"]]

# Definir a variável dependente e as independentes
y_name = "pib"
x_names = ["idhm", "pop"]

In [ ]:
# Criar matriz de pesos espaciais
w = weights.Queen.from_dataframe(data)

# Normalizar a matriz de pesos espaciais
w.transform = 'r'

# Calcular a estatística de Moran's I para o erro usando pysal
model = spreg.GM_Lag(data[y_name].values.reshape(-1, 1), data[x_names].values, w, name_y=y_name, name_x=x_names)
moran_resid = esda.moran.Moran(model.u, w)
print("Moran's I do erro:", moran_resid.I)

# Calcular a estatística de Moran's I para o resíduo usando pysal
moran_residuals = esda.moran.Moran(model.u_residuals, w)
print("Moran's I do resíduo:", moran_residuals.I)

# Calcular o teste de heterocedasticidade de Breusch-Pagan usando pysal
resid_std_err = np.sqrt(model.vm.diagonal())
lm_lag_std_err = np.sqrt(model.sig2ML)
test = spreg.diagnostics.het_breuschpagan(model.u_residuals, model.X, w, lm_lag_std_err, resid_std_err)
print("p-value do teste de Breusch-Pagan para heterocedasticidade:", test.pvalue)

# Ajustar o modelo de regressão espacial com GM-Lag
model_robust = spreg.GM_Lag(data[y_name].values.reshape(-1, 1), data[x_names].values, w, name_y=y_name, name_x=x_names, robust='hac', gwk=w)

# Imprimir os resultados do modelo ajustado
print(model_robust.summary)